In [1]:
import math

GRID_MIN = 0
GRID_MAX = 200

def is_obstacle(x, y):
    return (
        (40 <= x <= 70 and 40 <= y <= 70) or
        (90 <= x <= 130 and 120 <= y <= 160) or
        (150 <= x <= 180 and 50 <= y <= 90) or
        (60 <= x <= 100 and 150 <= y <= 190)
    )

def is_valid(x, y):
    return (
        GRID_MIN <= x <= GRID_MAX and
        GRID_MIN <= y <= GRID_MAX and
        not is_obstacle(x, y)
    )

MOVES = [
    (1, 0, 1), (-1, 0, 1), (0, 1, 1), (0, -1, 1),     #(dx, dy, cost)
    (1, 1, math.sqrt(2)), (1, -1, math.sqrt(2)),
    (-1, 1, math.sqrt(2)), (-1, -1, math.sqrt(2))
]

def get_neighbors(x, y):
    neighbors = []
    for dx, dy, cost in MOVES:
        nx, ny = x + dx, y + dy
        if is_valid(nx, ny):
            neighbors.append(((nx, ny), cost))
    return neighbors


In [2]:
import heapq

START = (0, 0)
GOAL = (200, 200)

def dijkstra(start, goal):
    # Priority queue
    pq = []
    heapq.heappush(pq, (0, start))

    # Distance dictionary
    dist = {start: 0}

    # Parent dictionary for path reconstruction
    parent = {}

    expanded_nodes = 0

    while pq:
        current_dist, current_node = heapq.heappop(pq)
        expanded_nodes += 1

        if current_node == goal:
            break

        # Skip outdated queue entries
        if current_dist > dist[current_node]:
            continue

        x, y = current_node

        # Explore neighbors
        for (nx, ny), cost in get_neighbors(x, y):
            new_dist = current_dist + cost

            if (nx, ny) not in dist or new_dist < dist[(nx, ny)]:
                dist[(nx, ny)] = new_dist
                parent[(nx, ny)] = current_node
                heapq.heappush(pq, (new_dist, (nx, ny)))

    # Reconstruct shortest path
    path = []
    node = goal
    while node != start:
        path.append(node)
        node = parent[node]
    path.append(start)
    path.reverse()

    return path, dist[goal], expanded_nodes
path, cost, expanded = dijkstra(START, GOAL)

print("Shortest path cost:", cost)
print("Number of expanded nodes:", expanded)
print("Path length:", len(path))


Shortest path cost: 301.002092041054
Number of expanded nodes: 40457
Path length: 232


In [3]:
import heapq
import math

def heuristic(node):
    x, y = node
    gx, gy = GOAL
    return math.sqrt((x - gx)**2 + (y - gy)**2)

def astar(start, goal):
    pq = []
    heapq.heappush(pq, (heuristic(start), 0, start))

    # Cost from start
    g_cost = {start: 0}

    # Parent dictionary
    parent = {}

    expanded_nodes = 0

    while pq:
        f, current_g, current_node = heapq.heappop(pq)
        expanded_nodes += 1

        if current_node == goal:
            break

        if current_g > g_cost[current_node]:
            continue

        x, y = current_node

        # Explore neighbors
        for (nx, ny), move_cost in get_neighbors(x, y):
            new_g = current_g + move_cost

            if (nx, ny) not in g_cost or new_g < g_cost[(nx, ny)]:
                g_cost[(nx, ny)] = new_g
                parent[(nx, ny)] = current_node
                f_cost = new_g + heuristic((nx, ny))
                heapq.heappush(pq, (f_cost, new_g, (nx, ny)))

    # Reconstruct path
    path = []
    node = goal
    while node != start:
        path.append(node)
        node = parent[node]
    path.append(start)
    path.reverse()

    return path, g_cost[goal], expanded_nodes
path_astar, cost_astar, expanded_astar = astar(START, GOAL)

print("A* path cost:", cost_astar)
print("A* expanded nodes:", expanded_astar)
print("\nHeuristic Comment:")
print(
    "The heuristic used is the Euclidean distance to the goal, which represents "
    "a lower-bound estimate of the remaining cost when diagonal movements are allowed. "
    "Therefore, it is reasonable to believe that the heuristic does not overestimate "
    "the true remaining cost."
)


A* path cost: 301.002092041054
A* expanded nodes: 22924

Heuristic Comment:
The heuristic used is the Euclidean distance to the goal, which represents a lower-bound estimate of the remaining cost when diagonal movements are allowed. Therefore, it is reasonable to believe that the heuristic does not overestimate the true remaining cost.


In [4]:
path_dij, cost_dij, expanded_dij = dijkstra(START, GOAL)
path_astar, cost_astar, expanded_astar = astar(START, GOAL)
print("Dijkstra:")
print("  Path length:", len(path_dij))
print("  Expanded nodes:", expanded_dij)

print("\nA*:")
print("  Path length:", len(path_astar))
print("  Expanded nodes:", expanded_astar)

print("\n"
    "Dijkstra is less efficient for large search spaces because it lacks direction.\nA* is more efficient due to heuristic guidance, reducing unnecessary exploration.It performs better in practice while maintaining optimality."


)


Dijkstra:
  Path length: 232
  Expanded nodes: 40457

A*:
  Path length: 232
  Expanded nodes: 22924

Dijkstra is less efficient for large search spaces because it lacks direction.
A* is more efficient due to heuristic guidance, reducing unnecessary exploration.It performs better in practice while maintaining optimality.
